# Import Library

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import numpy as np

sns.set_style("darkgrid")
d_order = ['N', 'NNE', 'NE', 'ENE', 
        'E', 'ESE', 'SE', 'SSE', 
        'S', 'SSW', 'SW', 'WSW', 
        'W', 'WNW', 'NW', 'NNW']

In [ ]:
readable_mapper = {
    'giaret': 'Giá rét',
    'lanh': 'Lạnh',
    'matme': 'Mát mẻ',
    'amap': 'Ấm áp',
    'nong': 'Nóng',
    'oia': 'Oi ả',
    'emdem': 'Êm đềm',
    'nhe': 'Nhẹ',
    'trungbinh': 'Trung bình',
    'manh': 'Mạnh',
    'khongmua': 'Không mưa',
    'nho': 'Nhỏ',
    'vua': 'Vừa',
    'to': 'To',
    'ratto': 'Rất to',
    'thap': 'Thấp',
    'cao': 'Cao',
    'ratcao': 'Rất cao',
    'trongxanh': 'Trong xanh',
    'comay': 'Có mây',
    'nhieumay': 'Nhiều mây',
    'baophu': 'Mây bao phủ',
    'mb': 'Miền Bắc',
    'dhntb': 'Duyên hải Nam Trung Bộ',
    'tn': 'Tây Nguyên',
    'mn': 'Miền Nam'
}

In [ ]:
temp_order = ['giaret', 'lanh', 'matme', 'amap', 'nong', 'oia']
wind_order = ['emdem', 'nhe', 'trungbinh', 'manh']
rain_order = ['khongmua', 'nho', 'vua', 'to', 'ratto']
cloud_order = ['trongxanh', 'comay', 'nhieumay', 'baophu']
humidi_order = ['thap', 'trungbinh', 'cao', 'ratcao']
pressure_order = ['thap', 'trungbinh', 'cao']

In [ ]:
def anno_number(ax, lim_ratio = 1.1, dis_ratio = 0.02, font = 11):
    max_height = max([p.get_height() for p in ax.patches])
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.0f}', (p.get_x() + p.get_width() / 2, p.get_height() + max_height * dis_ratio), fontsize = font, ha = 'center')
    ax.set_ylim(0, max_height * lim_ratio)

In [ ]:
vietnam_geo = json.load(open('../datasets/vietnam_state.geojson', 'r'))

In [ ]:
df = pd.read_csv("../datasets/weather_cleaned.csv")

# Data Exploring

In [ ]:
def wind_d_plot(data, ax0, ax1 = None, cmap = 'Blues', unit = 'count', datamin = None, datamax = None):
    '''
    data: dataframe with columns 'd' (means direction) and 'count' (of that direction, sometimes not the real count, but the statistic of other columns)
    ax0: ax to plot wind direction pie chart
    ax1: ax to plot colorbar
    '''
    data1 = pd.DataFrame(d_order, columns = ['d'])
    data1['order'] = data1['d'].apply(lambda x: d_order.index(x))

    if not datamin: datamin = data['count'].min()
    if not datamax: datamax = data['count'].max()

    data = data1.join(data.set_index('d'), on = 'd', how = 'left', rsuffix = 'r').fillna(0)

    data['order'] = data['d'].apply(lambda x: d_order.index(x))
    data = data.sort_values(by = 'order')

    theme = plt.get_cmap(cmap)
    ax0.set_prop_cycle('color', theme((data['count'] - datamin)/(datamax- datamin)))
    ax0.pie(
        [1] * 16, labels = data['d'], startangle = 101.25, counterclock = False,
        wedgeprops = {'linewidth': 0.8, 'edgecolor': 'white'}, 
        textprops = {'ha': 'center', 'va': 'center'}, labeldistance = 1.2
    )

    if ax1:
        gradient = np.tile(np.linspace(1, 0, 256).reshape(-1, 1), (1, 10))
        # gradient = np.vstack((gradient, gradient))

        ax1.imshow(gradient, aspect = 'auto', cmap = cmap)
        ax1.grid(visible = False)
        ax1.set_xticks([])
        ax1.set_yticks(np.linspace(0, 255, 6))
        ax1.set_yticklabels(f'{i:.0f}' for i in np.linspace(datamax, datamin, 6))
        ax1.yaxis.tick_right()
        ax1.set_xlabel(unit)
        ax1.xaxis.set_label_position('top')

In [ ]:
fig, axes = plt.subplots(ncols = 4, nrows = 4, figsize = (15, 15))

#######################----MienBac----##########################
data = df[
    (df['month'].isin([1, 2, 3])) & (df['region'] == 'mb')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[0, 0], cmap = 'RdYlBu_r')

data = df[
    (df['month'].isin([4, 5, 6])) & (df['region'] == 'mb')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[0, 1], cmap = 'RdYlBu_r')

data = df[
    (df['month'].isin([7, 8, 9])) & (df['region'] == 'mb')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[0, 2], cmap = 'RdYlBu_r')

data = df[
    (df['month'].isin([10, 11, 12])) & (df['region'] == 'mb')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[0, 3], cmap = 'RdYlBu_r')

#######################----MienTrung----##########################
data = df[
    (df['month'].isin([1, 2, 3])) & (df['region'] == 'dhntb')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[1, 0], cmap = 'RdYlBu_r')

data = df[
    (df['month'].isin([4, 5, 6])) & (df['region'] == 'dhntb')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[1, 1], cmap = 'RdYlBu_r')

data = df[
    (df['month'].isin([7, 8, 9])) & (df['region'] == 'dhntb')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[1, 2], cmap = 'RdYlBu_r')

data = df[
    (df['month'].isin([10, 11, 12])) & (df['region'] == 'dhntb')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[1, 3], cmap = 'RdYlBu_r')

#######################----TayNguyen----##########################
data = df[
    (df['month'].isin([1, 2, 3])) & (df['region'] == 'tn')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[2, 0], cmap = 'RdYlBu_r')

data = df[
    (df['month'].isin([4, 5, 6])) & (df['region'] == 'tn')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[2, 1], cmap = 'RdYlBu_r')

data = df[
    (df['month'].isin([7, 8, 9])) & (df['region'] == 'tn')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[2, 2], cmap = 'RdYlBu_r')

data = df[
    (df['month'].isin([10, 11, 12])) & (df['region'] == 'tn')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[2, 3], cmap = 'RdYlBu_r')

#######################----MienNam----##########################
data = df[
    (df['month'].isin([1, 2, 3])) & (df['region'] == 'mn')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[3, 0], cmap = 'RdYlBu_r')

data = df[
    (df['month'].isin([4, 5, 6])) & (df['region'] == 'mn')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[3, 1], cmap = 'RdYlBu_r')

data = df[
    (df['month'].isin([7, 8, 9])) & (df['region'] == 'mn')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[3, 2], cmap = 'RdYlBu_r')

data = df[
    (df['month'].isin([10, 11, 12])) & (df['region'] == 'mn')
].groupby(by = ['wind_d'])['max'].agg('count').reset_index()
data.columns = ['d', 'count']

wind_d_plot(data, axes[3, 3], cmap = 'RdYlBu_r')

axes[0, 0].set_title('Tháng 1, 2, 3', pad = 20)
axes[0, 1].set_title('Tháng 4, 5, 6', pad = 20)
axes[0, 2].set_title('Tháng 7, 8, 9', pad = 20)
axes[0, 3].set_title('Tháng 10, 11, 12', pad = 20)

axes[0, 0].set_ylabel('Miền Bắc', rotation = 0, labelpad = 50, fontsize = 12)
axes[1, 0].set_ylabel('Duyên hải\nNam Trung Bộ', rotation = 0, labelpad = 50, fontsize = 12)
axes[2, 0].set_ylabel('Tây Nguyên', rotation = 0, labelpad = 50, fontsize = 12)
axes[3, 0].set_ylabel('Miền Nam', rotation = 0, labelpad = 50, fontsize = 12)

Nhìn chung hướng gió có sự khác biệt ở các vùng miền khác nhau vào các khoảng thời gian khác nhau, 

- Miền Bắc: chủ yếu các hướng Đông, trải từ Đông Bắc tới Nam Đông Nam
- Duyên hải Nam Trung Bộ: Đông Bắc từ tháng 10 tới tháng 3 năm sau, từ Đông Nam tới Tây Nam vào tháng 4 tới tháng 9
- Tây Nguyên: Đông Đông Bắc từ tháng 10 tới tháng 3 năm sau, Tây Nam từ tháng 4 tới tháng 9
- Miền Nam: Đống, Đông Nam từ tháng 10 tới tháng 3 năm sau, Tây Nam từ tháng 4 tới tháng 9

Ngoại trừ miền Bắc hướng gió thay đổi chủ yếu ở hướng Đông, các vùng còn lại có 2 hướng gió chủ yếu:
- Đông, Đông Bắc: từ tháng 10 tới tháng 3 năm sau
- Nam, Tây Nam: từ tháng 4 tới tháng 9

Một đặc điểm chung nữa của các vùng là ghi nhận rất ít gió thổi từ các hướng Tây Bắc